# Nuevos listados

En esta notebook, vamos a generar los nuevos listados de palabras...

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["Palabra Candidata"] = analizadas["Palabra Candidata"].astype('str')

In [2]:
palabras_ya_analizadas = analizadas[analizadas['Palabra Candidata'] != 'nan'].index

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


- Armar listados para cada una de las métricas acá

## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 2]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

Calculemos el valor de la información de cada palabra.

In [4]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log(23) - df["entropy_palabras"]
df["delta_personas"] = np.log(23) - df["entropy_personas"]

df["log_cantidad"] = np.log(1+df.cantPalabra)
df["log_personas"] = np.log(1+df.cantUsuariosTotal)

df["norm_cantidad"] = df["log_cantidad"]  / df["log_cantidad"].max()
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [5]:
def analizar_palabras_restantes(n):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    for nombre, this_df in [("Palabras", df_palabras), ("Personas", df_personas), ("PalPer", df_pp)]: 
        restantes = {p for p in this_df.iloc[:n].index if p not in palabras_ya_analizadas}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("{}  => faltan etiquetar {} palabras".format(nombre, len(restantes)))

    print("Total = {}".format(total))
    print("Total (sin rep) = {}".format(len(todas_las_palabras)))

analizar_palabras_restantes(1000)
analizar_palabras_restantes(5000)



Primeras 1000 palabras
Palabras  => faltan etiquetar 219 palabras
Personas  => faltan etiquetar 0 palabras
PalPer  => faltan etiquetar 0 palabras
Total = 219
Total (sin rep) = 219
Primeras 5000 palabras
Palabras  => faltan etiquetar 1602 palabras
Personas  => faltan etiquetar 653 palabras
PalPer  => faltan etiquetar 1057 palabras
Total = 3312
Total (sin rep) = 1820


## Usando cálculo viejo


In [6]:
from contrastes.information_value import simulated_shuffled_entropy

In [7]:
fn = lambda ws: simulated_shuffled_entropy(int(ws), len(cant_palabras))
df["sh_entropy_palabras"] = df.cantPalabra.apply(fn)
df["sh_entropy_personas"] = df.cantUsuariosTotal.apply(fn)


In [8]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cantPalabra)
df["log_personas"] = np.log(df.cantUsuariosTotal)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())
    
df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:

analizar_palabras_restantes(1000)
analizar_palabras_restantes(5000)

Primeras 1000 palabras
Palabras  => faltan etiquetar 213 palabras
Personas  => faltan etiquetar 0 palabras
PalPer  => faltan etiquetar 327 palabras
Total = 540
Total (sin rep) = 540
Primeras 5000 palabras
Palabras  => faltan etiquetar 1827 palabras
Personas  => faltan etiquetar 756 palabras
PalPer  => faltan etiquetar 3741 palabras
Total = 6324
Total (sin rep) = 6211
